# Working with raw precipitation versus Synoptic's derived precipitation product

Here's an example that illustrates some of the challenges of working with real time precipitation data from stations across many different networks.

In [20]:
# First the basics
import urllib.request as req
import json
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import data_table
data_table.enable_dataframe_formatter()
%matplotlib inline

def return_station_df(data, service):
    """Build pandas dataframes for data and metadata using json response from 
        requests to Time Series, Nearest, and Latest services

    Parameters:
        data: dict, json response from API request
        service: str, Synoptic web service requested
    
    Returns:
        data_df: pandas DataFrame, data return from all station
        meta_df: pandas DataFrame, station metadata
    """
    dattim_format = '%Y-%m-%d %H:%M'
    meta_list = []
    # We iterate over the list of stations
    for i in range(len(data)):
        # Append station metadata to a grand list that we'll convert to a df
        stid = data[i]['STID']
        mnet_id = data[i]['MNET_ID']
        try:
            lon = float(data[i]['LONGITUDE'])
        except TypeError:
            lon = None
        try:
            lat = float(data[i]['LATITUDE'])
        except TypeError:
            lat = None
        try:
            elev = float(data[i]['ELEVATION'])
        except TypeError:
            elev = None
        meta_list.append([stid, mnet_id, lon, lat, elev])

        # Create a multi-index object to attach to the data df
        data_out = data[i]['OBSERVATIONS'].copy()
        if service == 'timeseries':
            datetime = pd.to_datetime(data_out['date_time'], format=(dattim_format))
            del data_out['date_time']
            multi_index = pd.MultiIndex.from_product([[stid], datetime],
                                                     names=["stid", "dattim"])
        else:
            datetime = pd.to_datetime(data_out[list(data_out.keys())[0]]['date_time'], format=(dattim_format))
            for key in data_out:
                data_out.update({key: data_out[key]['value']})
            multi_index = pd.MultiIndex.from_arrays([[stid], [datetime]],
                                                    names=["stid", "dattim"])

        # Build the data df, concatenating as needed
        if i == 0:
            data_df = pd.DataFrame(data_out, index=multi_index)
        else:
            data_df = pd.concat([data_df, pd.DataFrame(data_out, index=multi_index)], axis=0)

    #Build metadata dataframe from list
    meta_df = pd.DataFrame(meta_list, columns=["stid", "mnet_id", "lon", "lat", "elev"])
    meta_df.set_index('stid', inplace=True)

    # Sort the resulting data dataframe by time
    data_df.sort_index(inplace=True)

    return data_df, meta_df

In [21]:
def make_api_request(url, api_args):
    """Build the api request from the url and api_args, make the request, and parse 
    the json return to a dictionary

    Parameters:
        url: str, url of the api endpoint
        api_args: dict, api arguments
    
    Returns:
        output: dict, api request response 
    """
    # Append the api arguments on to the url
    for argument, value in api_args.items():
        url = url + '&' + argument + '=' + value

    # Make the api request
    print(f"API request: {url}")
    with req.urlopen(url) as response:
        body = response.read()

    # parse the json response. 
    try:
        output = json.loads(body)
    except:
        decoded_body = body.decode('latin1')
        output = json.loads(decoded_body)

    return output

# Example case: Missoula, MT

For this example we'll focus on a wet weekend in Missoula, MT back in June. Let's request data from all the stations in Missoula County.

In [22]:
token = ''
url = 'https://api.synopticdata.com/v2/stations/timeseries?'
api_args = {'state': 'mt',
            'county': 'missoula',
            'start': '202206100000',
            'end': '202206130000',
            'obtimezone': 'local',
            'token': token}

In [ ]:
data = make_api_request(url, api_args)
data['SUMMARY']

# The challenge: stations can report precipitation in a number of different formats

Let's see how many different precip variables are returned by inspecting the `UNITS` dictionary

In [ ]:
data['UNITS']

We'll create our data and metadata dataframes and explore in more detail the stations reporting different precip variable types

In [ ]:
data_df, meta_df = return_station_df(data['STATION'], 'timeseries')

Let's create a dataframe with stations reporting `precip_accum` values

In [ ]:
precip_accum_df = data_df['precip_accum_set_1'].dropna()
precip_accum_df


And a dataframe with stations reporting `precip_accum_since_local_midnight`

In [ ]:
precip_mid_df = data_df['precip_accum_since_local_midnight_set_1'].dropna()
precip_mid_df


And one more data frame with stations reporting `precip_accum_one_hour`:

In [ ]:
precip_1hr_df = data_df['precip_accum_one_hour_set_1'].dropna()
precip_1hr_df

We can demonstrate the challenges of working with precip by plotting timeseries from a station reporting one of each of these values

In [ ]:
fig1 = plt.figure(1,figsize=(8,16))
ax1a = fig1.add_subplot(311)
ax1a.plot(precip_accum_df.loc['BLMM8'].index, precip_accum_df.loc['BLMM8'])
ax1a.set_title('BLMM8')

ax1b = fig1.add_subplot(312)
ax1b.plot(precip_mid_df.loc['C4884'].index, precip_mid_df.loc['C4884'])
ax1b.set_title('C4884')

ax1c = fig1.add_subplot(313)
ax1c.plot(precip_1hr_df.loc['KMSO'].index, precip_1hr_df.loc['KMSO'])
ax1c.set_title('KMSO')

# One solution: derived precipitation

Synoptic processes precipitation observations in real time, building a derived precipitation product that collapses different precipitation reporting into a consistent format. The derived precipitation product is accessed from the Timeseries endpoint by setting the `precip` argument.

In [ ]:
url = 'https://api.synopticdata.com/v2/stations/timeseries?'
api_args2 = {'stid': 'KMSO,C4884,BLMM8',
            'start': '202206100000',
            'end': '202206130000',
            'obtimezone': 'local',
            'precip':'1',
            'token': token}

In [ ]:
data2 = make_api_request(url, api_args2)
data2['SUMMARY']

In [ ]:
data2['UNITS']

In [ ]:
data_df2, meta_df2 = return_station_df(data2['STATION'], 'timeseries')

In [ ]:
data_df2.columns

Let's plot up the resulting precip accumulations

In [ ]:
fig2 = plt.figure(2,figsize=(8,16))
ax2a = fig2.add_subplot(311)
ax2a.plot(data_df2.loc['BLMM8'].index, data_df2.loc['BLMM8','precip_accumulated_set_1d'])
ax2a.set_title('BLMM8')

ax2b = fig2.add_subplot(312)
ax2b.plot(data_df2.loc['C4884'].index, data_df2.loc['C4884','precip_accumulated_set_1d'])
ax2b.set_title('C4884')

ax2c = fig2.add_subplot(313)
ax2c.plot(data_df2.loc['KMSO'].index, data_df2.loc['KMSO','precip_accumulated_set_1d'])
ax2c.set_title('KMSO')

Despite the different reporting formats among the 3 stations, the measured precipitation shows a similar pattern over the weekend.

# Exercise: Pick a handful of stations around CA from the recent Atmospheric River Events and plot up a time series of derived precipitation. How much precip has fallen? How much variability is there between stations?